<a href="https://colab.research.google.com/github/takaito1/EAS6305_F24/blob/main/week2/calc_MLD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mixed layer depth

    - The goal of this homework is to use the CTD profile data to calculate sigma-theta and mixed layer depth (MLD)
    - Use the sigma-theta profile to compute the MLD

In [ ]:
! pip install gsw

In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import gsw
import pandas as pd

### (a) Read BATS data and calculate sigma-theta

In [ ]:
# Step 1 download data from BATS
df = pd.read_csv('https://datadocs.bco-dmo.org/file/m7zA3onuPG01JD/3918_v10_bats_ctd_2020-2024.csv')
df.head() # display the first 5 rows

In [ ]:
# define headers first (since the excel sheet doesn't have one)
headers = df.columns
print(headers)

In [ ]:
# extract a specific parameter from specific cast / cruise
new_df = df.loc[(df.Cruise_num==10370)&(df.Cast==1)]
T = new_df['Temperature'].to_numpy()
S = new_df['Salinity'].to_numpy()
P = new_df['Pressure'].to_numpy()
lon = new_df['Longitude_deployed'].to_numpy()
lat = new_df['Latitude_deployed'].to_numpy()

In [ ]:
# Step 2 calculate sigma-theta
sa = gsw.SA_from_SP(S,P,lon,lat) # calculate absolute salinity
ct = gsw.CT_from_t(sa,T,P) # calculate conservative temperature
#
sigma0 = gsw.sigma0(sa,ct) # calculate sigma-theta

In [ ]:
# plot
fig=plt.figure(figsize=(10,5))
ax=fig.subplots(1,3)
plt.subplots_adjust(wspace=0.5)
#
ax[2].plot(sigma0,P,label='sigma-theta')
ax[2].set_xlabel('sigma-theta, kg/m3')
ax[2].set_ylabel('pressure, dbar')
ax[2].set_ylim(1200,0)
#
ax[0].plot(sa,P,label='SA')
ax[0].set_xlabel('absolute salinity, g/kg')
ax[0].set_ylabel('pressure, dbar')
ax[0].set_ylim(1200,0)
#
ax[1].plot(ct,P,label='CT')
ax[1].set_xlabel('conservative temperature, deg C')
ax[1].set_ylabel('pressure, dbar')
ax[1].set_ylim(1200,0)
plt.show()

### (b) Determine mixed layer depth using sigma-theta profile

In [ ]:
# First calculate the surface value
sig0_surf = min(sigma0)
print(f'Surface sigma-theta = {np.round(sig0_surf,2)} kg/m3')

In [ ]:
# Start the loop
#
###### define density threshold
sig0_crit =
######

######
# mixed layer definition algorithm
for n,pres in enumerate(P):
    offset =
    if offset < sig0_crit:
        mld = P[n]
######

print(f'Mixed layer depth is {np.round(mld,2)} dbar level')

In [ ]:
# plot
fig=plt.figure(figsize=(10,5))
ax=fig.subplots(1,3)
plt.subplots_adjust(wspace=0.5)
#
ax[2].plot(sigma0,P,label='sigma-theta')
ax[2].plot([min(sigma0), max(sigma0)],[mld, mld],'r',label='mixed layder depth')
ax[2].set_xlabel('sigma-theta, kg/m3')
ax[2].set_ylabel('pressure, dbar')
ax[2].set_ylim(200,0)
#
ax[0].plot(sa,P,label='SA')
ax[0].plot([min(sa), max(sa)],[mld, mld],'r',label='mixed layder depth')
ax[0].set_xlabel('absolute salinity, g/kg')
ax[0].set_ylabel('pressure, dbar')
ax[0].set_ylim(200,0)
#
ax[1].plot(ct,P,label='CT')
ax[1].plot([min(ct), max(ct)],[mld, mld],'r',label='mixed layder depth')
ax[1].set_xlabel('conservative temperature, deg C')
ax[1].set_ylabel('pressure, dbar')
ax[1].set_ylim(200,0)
plt.show()
